In [26]:
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models
from tensorflow.keras.datasets.fashion_mnist import load_data
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [30]:
np.random.seed(1997)
tf.random.set_seed(1997)

In [31]:
(X_train, y_train), (X_test, y_test) = load_data()

In [32]:
X_train = X_train.reshape(len(X_train), -1)
y_train = y_train.reshape(len(y_train), -1)
X_test = X_test.reshape(len(X_test), -1)
y_test = y_test.reshape(len(y_test), -1)

In [33]:
X_train = X_train.astype(float)/255
X_test = X_test.astype(float)/255

In [34]:
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [41]:
class DNN(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.layer1 = models.Sequential([
            layers.Dense(128),
            layers.Dropout(0.3),
            layers.BatchNormalization(),
            layers.ReLU()
        ])
        self.layer2 = models.Sequential([
            layers.Dense(2048),
            layers.Dropout(0.3),
            layers.BatchNormalization(),
            # BatchNormalization => 장점을 찾아보기
            layers.ReLU()
        ])
        self.layer3 = models.Sequential([
            layers.Dense(10),
            layers.Softmax()
        ])
        
    def call(self,x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x

In [42]:
DnnModel = DNN()
DnnModel.build(input_shape = (512, 784))
#메모리 소비가 200과 256이 동일 그러므 256사용

In [43]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
DnnModel.compile(optimizer=optimizer, loss = 'categorical_crossentropy', metrics=['accuracy'])

In [44]:
#콜백함수
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
def scheduler(epoch,lr):
    tf.print(f'learning rate : {lr:5f}')
    if epoch < 5:
        return lr
    elif epoch < 10:
        return lr * tf.math.exp(-0.1)
    else:
        return lr * tf.math.exp(-0.01)
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler) 
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath = 'tmp_checkpoint.ckpt', 
                                                save_weights_only=True,
                                                save_best_only=True,
                                                monitor = 'val_loss',
                                                verbose=1)




In [46]:
DnnModel.fit(X_train, y_train, callbacks=[earlystopping, lr_scheduler, checkpoint], batch_size=128, epochs=20, validation_data=(X_test, y_test))

Train on 60000 samples, validate on 10000 samples
learning rate : 0.005827
Epoch 1/20
59392/60000 [============================>.] - ETA: 0s - loss: 0.2372 - accuracy: 0.9106
Epoch 00001: val_loss improved from 0.33967 to 0.32851, saving model to tmp_checkpoint.ckpt
60000/60000 [==============================] - 3s 55us/sample - loss: 0.2371 - accuracy: 0.9105 - val_loss: 0.3285 - val_accuracy: 0.8843
learning rate : 0.005827
Epoch 2/20
59904/60000 [============================>.] - ETA: 0s - loss: 0.2294 - accuracy: 0.9131
Epoch 00002: val_loss did not improve from 0.32851
60000/60000 [==============================] - 3s 55us/sample - loss: 0.2296 - accuracy: 0.9130 - val_loss: 0.3465 - val_accuracy: 0.8801
learning rate : 0.005827
Epoch 3/20
59904/60000 [============================>.] - ETA: 0s - loss: 0.2251 - accuracy: 0.9145
Epoch 00003: val_loss did not improve from 0.32851
60000/60000 [==============================] - 3s 56us/sample - loss: 0.2254 - accuracy: 0.9145 - val_los

In [49]:
DnnModel.evaluate(X_test,y_test)

10000/10000 [==============================] - 0s 45us/sample - loss: 0.3354 - accuracy: 0.8840


[0.3354127689957619, 0.884]